# Cirrus fractions for obs climatology
directly related to conv vs. in situ

using 

1. **lidar only vs. lidar & radar**
2. Brightness temperature thresholds (to match DYAMOND)


* at +500m
* at +1000m
* at +500m and +1000 m

## TODO: save as dictionaries?
probably will want to for the paper if we're making a table...

In [1]:
import sys
sys.path.append("/home/b/b380887/cold-point-overshoot/python_scripts")
import dask
import pickle

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker
import bin_obs_overshoot as boo

from dask.diagnostics import ProgressBar
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


In [2]:
season = "JJA"


In [3]:
if season == "DJF":
    region_list = ["AMZ", "SPC", "ECP", "IOS"]
    year_list = [2007, 2008, 2009, 2010]
    
elif season == "JJA":
    region_list = ["AFR", "WPC", "ECP", "IOE"]
    year_list = [2007, 2008, 2009, 2010] # TODO: add 2006 when done
    
years_sorted = list(sorted(year_list))
years_str = str(years_sorted[0]) + "-" + str(years_sorted[-1])
print(years_str)


2007-2010


### Get the climo datasets for lidar-only and radar+lidar ice at +500m and +1000m

In [4]:
file_path = "/work/bb1153/b380887/big_obs_climo/"


In [5]:
ds_dict = {}
variables = ["iwc_a500", "iwc_a1000", "iflag_a500", "iflag_a1000"]

for region in region_list:
    dar_list = [[]]*len(year_list)

    for i, year in enumerate(year_list):
        if region == "SPC":
            ds_all_1 = xr.open_dataset(file_path + "{s}/DARDAR_cp_relative_iwc_{s}{y}_{r}1.nc".format(s=season, y=year, r=region))
            ds_all_2 = xr.open_dataset(file_path + "{s}/DARDAR_cp_relative_iwc_{s}{y}_{r}2.nc".format(s=season, y=year, r=region))            
            dar_list[i] = xr.concat([ds_all_1, ds_all_2], dim="time")
        else:
            dar_list[i] = xr.open_dataset(file_path + "{s}/DARDAR_cp_relative_iwc_{s}{y}_{r}.nc".format(s=season, y=year, r=region))

    ds = xr.concat(dar_list, dim="time")
    ds_dict[region] = ds[variables]
    print(region + " done")
                         

AFR done
WPC done
ECP done
IOE done


#### Instrument flag:
* 1 = lidar only
* 3 = radar + lidar

##### NOTE: 
The means are **IWC**, not **qi** - won't match up with binned plot unless you convert htose with the reference densities

In [6]:
def get_cirrus_stats(ds, region, iwc_min=None):
    """ Get frequency of retrievals at each level
    """
    # iwc at each level, conditioned by instrument flag
    iwc_a500_lid = ds["iwc_a500"].where(ds["iflag_a500"] == 1.0)
    iwc_a500_both = ds["iwc_a500"].where(ds["iflag_a500"] == 3.0)
    iwc_a1000_lid = ds["iwc_a1000"].where(ds["iflag_a1000"] == 1.0)
    iwc_a1000_both = ds["iwc_a1000"].where(ds["iflag_a1000"] == 3.0)
    
    # exclude retrievals with no ice detected if requested
    # (makes very little difference because iflag already excludes almost all of those)
    if iwc_min is not None:
        iwc_a500_lid = iwc_a500_lid.where(iwc_a500_lid > iwc_min)
        iwc_a500_both = iwc_a500_both.where(iwc_a500_both > iwc_min)
        iwc_a1000_lid = iwc_a1000_lid.where(iwc_a1000_lid > iwc_min)
        iwc_a1000_both = iwc_a1000_both.where(iwc_a1000_both > iwc_min)

    # total number of retrievals at each level
    tot_500 = ds["iwc_a500"].count()
    tot_1000 = ds["iwc_a1000"].count()

    # fractions of retrievals from each instrument at each level
    lid_frac_500 = iwc_a500_lid.count() / tot_500
    both_frac_500 = iwc_a500_both.count() / tot_500
    lid_frac_1000 = iwc_a1000_lid.count() / tot_1000
    both_frac_1000 = iwc_a1000_both.count() / tot_1000

    # mean IWC from each instrument at each level
    lid_mean_500 = iwc_a500_lid.mean()
    both_mean_500 = iwc_a500_both.mean()
    lid_mean_1000 = iwc_a1000_lid.mean()
    both_mean_1000 = iwc_a1000_both.mean()

    # same as above, but for +500 and +1000 levels combined
    lid_frac_51 = (iwc_a500_lid.count() + iwc_a1000_lid.count()) / (tot_500 + tot_1000)
    both_frac_51 = (iwc_a500_both.count() + iwc_a1000_both.count()) / (tot_500 + tot_1000)
    lid_mean_51 = xr.concat([iwc_a500_lid, iwc_a1000_lid], dim="time").mean() / (tot_500 + tot_1000)
    both_mean_51 = xr.concat([iwc_a500_both, iwc_a1000_both], dim="time").mean() / (tot_500 + tot_1000)

    print("{r}, +500m: lidar only={l:.2f}%, both={b:.3f}%".format(l=lid_frac_500.values*100., b=both_frac_500.values*100., r=region))
    print("{r}, +500m: lidar only={l:.1e} kg/m3, both={b:.1e} kg/m3".format(l=lid_mean_500.values, b=both_mean_500.values, r=region))
    print("{r}, +1000m: lidar only={l:.2f}%, both={b:.3f}%".format(l=lid_frac_1000.values*100., b=both_frac_1000.values*100., r=region))
    print("{r}, +1000m: lidar only={l:.1e} kg/m3, both={b:.1e} kg/m3".format(l=lid_mean_1000.values, b=both_mean_1000.values, r=region))

        

In [7]:
# DJF
for region in region_list:
    get_cirrus_stats(ds_dict[region], region, iwc_min=None)
    print("\n")


AMZ, +500m: lidar only=2.56%, both=0.012%
AMZ, +500m: lidar only=1.2e-06 kg/m3, both=4.9e-05 kg/m3
AMZ, +1000m: lidar only=0.23%, both=0.001%
AMZ, +1000m: lidar only=1.1e-06 kg/m3, both=1.5e-05 kg/m3


SPC, +500m: lidar only=2.42%, both=0.040%
SPC, +500m: lidar only=1.1e-06 kg/m3, both=5.2e-05 kg/m3
SPC, +1000m: lidar only=0.13%, both=0.001%
SPC, +1000m: lidar only=8.5e-07 kg/m3, both=5.6e-05 kg/m3


ECP, +500m: lidar only=0.41%, both=0.000%
ECP, +500m: lidar only=1.1e-06 kg/m3, both=1.7e-05 kg/m3
ECP, +1000m: lidar only=0.01%, both=0.000%
ECP, +1000m: lidar only=6.0e-07 kg/m3, both=nan kg/m3


IOS, +500m: lidar only=0.89%, both=0.016%
IOS, +500m: lidar only=1.3e-06 kg/m3, both=7.2e-05 kg/m3
IOS, +1000m: lidar only=0.03%, both=0.003%
IOS, +1000m: lidar only=1.5e-06 kg/m3, both=2.0e-05 kg/m3




In [7]:
# JJA
for region in region_list:
    get_cirrus_stats(ds_dict[region], region, iwc_min=None)
    print("\n")


AFR, +500m: lidar only=1.32%, both=0.033%
AFR, +500m: lidar only=1.8e-06 kg/m3, both=8.6e-05 kg/m3
AFR, +1000m: lidar only=0.10%, both=0.004%
AFR, +1000m: lidar only=2.3e-06 kg/m3, both=5.9e-05 kg/m3


WPC, +500m: lidar only=1.54%, both=0.022%
WPC, +500m: lidar only=1.8e-06 kg/m3, both=4.7e-05 kg/m3
WPC, +1000m: lidar only=0.08%, both=0.001%
WPC, +1000m: lidar only=1.4e-06 kg/m3, both=1.9e-05 kg/m3


ECP, +500m: lidar only=0.40%, both=0.002%
ECP, +500m: lidar only=1.7e-06 kg/m3, both=4.4e-05 kg/m3
ECP, +1000m: lidar only=0.01%, both=0.000%
ECP, +1000m: lidar only=1.3e-06 kg/m3, both=3.0e-05 kg/m3


IOE, +500m: lidar only=0.57%, both=0.011%
IOE, +500m: lidar only=2.2e-06 kg/m3, both=3.6e-05 kg/m3
IOE, +1000m: lidar only=0.02%, both=0.000%
IOE, +1000m: lidar only=1.5e-06 kg/m3, both=2.0e-05 kg/m3


